# Car Prices Data Science Learning

In this project we use dataset from [Kaggle](https://www.kaggle.com/datasets/syedanwarafridi/vehicle-sales-data).

## Imports and loading data

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
from sklearn.preprocessing import MinMaxScaler
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import geopandas as gpd

In [ ]:
df = pd.read_csv('car_prices.csv')
print(df.head())

In [ ]:
print(df.info())

In [ ]:
print(df.describe())

## Clearing

In [ ]:
nan_counts = df.isna().sum()
print(nan_counts)

In [ ]:
df['model'].value_counts(dropna=False)

In [ ]:
df.nunique()

In [ ]:
print(df['color'].unique())

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.loc[df['color'] != '—']

In [ ]:
df = df.dropna()
print(df.isna().sum())

In [ ]:
df.dtypes

## Scaling (Normalization)

In [ ]:
df1 = df[['year', 'condition', 'odometer', 'mmr', 'sellingprice']]
scaler = MinMaxScaler()
scaler.fit(df1)
print(scaler.data_max_)
print(scaler.transform(df1))

scaled_data = scaler.fit_transform(df1)

df2 = pd.DataFrame(scaled_data, columns=df1.columns)

print(df2)


## Data Visualization (Matplotlib)

In [ ]:
df['color'].value_counts().plot.bar()

In [ ]:
counts = df['make'].value_counts()
top_counts = counts.head(20)
top_counts.plot.bar()

In [ ]:
df_ford = df.loc[df['make'] == 'Ford']
ford_colors = df_ford['color'].value_counts()

df_chev = df.loc[df['make'] == 'Chevrolet']
chev_colors = df_chev['color'].value_counts()

colors_comparison = pd.DataFrame(data={'Ford': ford_colors, 'Chevrolet': chev_colors})
colors_comparison.plot.bar()
# print(colors_comparison)

## Data Visualization (Seaborn)

In [ ]:
df_sb = df[['year', 'make', 'sellingprice']]
selected_makes = ['Chevrolet', 'Lexus', 'Mazda']
df_sb = df_sb[df_sb['make'].isin(selected_makes)]
df_avg_price = df_sb.groupby(['make', 'year'])['sellingprice'].mean().reset_index()
df_avg_price

In [ ]:
sb.set_style("whitegrid")

plt.figure(figsize=(12, 8))
sb.pointplot(data=df_avg_price, x='year', y='sellingprice', hue='make', dodge=False, markers='o', linestyles='--')
plt.title('Point Plot for Sales Prices by Year and Brand')
plt.xlabel('Year')
plt.ylabel('Selling Price')
plt.xticks(rotation=45)
plt.legend(title='Make', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Pie charts

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], subplot_titles=('Transmission type', 'Brand'))

fig.add_trace(go.Pie(values=df['transmission'].value_counts(), 
                     labels=df['transmission'].unique(),
                     marker=dict(colors=px.colors.sequential.Bluered),
                     textposition='inside',
                     textinfo='percent+label+value',
                     textfont_size=12,
                     showlegend=False),
                     1, 1)

fig.add_trace(go.Pie(values=df['make'].value_counts(), 
                     labels=df['make'].unique(),
                     marker=dict(colors=px.colors.sequential.RdBu),
                     textposition='inside',
                     textinfo='percent+label',
                     textfont_size=12,
                     showlegend=True),
                     1, 2)


fig.show()


## Scatter plot

In [ ]:
sb.scatterplot(df, x="odometer", y="sellingprice")
plt.xlabel("Odometer [km]")
plt.ylabel("Selling Price[$]")
plt.title("Selling Price vs. Odometer Reading")
plt.grid(True)

plt.show()

In [ ]:
sb.scatterplot(df, x="mmr", y="sellingprice")
plt.xlabel("MMR")
plt.ylabel("Selling Price[$]")
plt.title("Selling Price vs. MMR")
plt.grid(True)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

sb.scatterplot(df, x="mmr", y="sellingprice", ax=ax[0])
sb.scatterplot(df, x="odometer", y="sellingprice", ax=ax[1])

plt.xlabel("MMR")
plt.ylabel("Selling Price[$]")
plt.xlabel("Odometer [km]")
plt.ylabel("Selling Price[$]")
plt.suptitle("Selling Price vs. MMR and Odometer")
plt.tight_layout()

plt.show()

## Geo charts


In [ ]:
url = "shapefiles/States_shapefile.shp"
gdf_states = gpd.read_file(url)

df_copy = df.copy()

df_copy.loc[:, 'state'] = df['state'].str.upper()

states = df_copy['state'].value_counts()
all_states = gdf_states['State_Code'].unique()

brand_counts = df_copy.groupby(['state', 'make']).size().reset_index(name='counts')

most_popular_brand = brand_counts.groupby('state').apply(lambda x: x.loc[x['counts'].idxmax()]).reset_index(drop=True)

merged_data = gdf_states.merge(most_popular_brand, left_on='State_Code', right_on='state', how='left')
merged_data['make'] = merged_data['make']

fig, ax = plt.subplots(1, 1, figsize=(20, 20))
gdf_states.boundary.plot(ax=ax, aspect=1, linewidth=.8, color='black')
merged_data.plot(column='make', legend=True, cmap='Set3', ax=ax, aspect=1)

plt.title('Most popular brand in each state')
plt.show()